In [1]:
# Import Library
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
DATA_DIR = '../data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

In [3]:
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
    # Train set
    # Image
    train_x = train_x.reshape([train_x.shape[0], -1])
    train_x = train_x.astype('float32') / 255
    # Label
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)

    # Test set
    # Image
    test_x = test_x.reshape([test_x.shape[0], -1])
    test_x = test_x.astype('float32') / 255
    # Label
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)
      
    return (train_x, train_y), (test_x, test_y)

In [4]:
(train_x, train_y), (test_x, test_y) = mnist_load()

buffer_size = train_x.shape[0] + test_x.shape[0]
dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(buffer_size).repeat().batch(MINIBATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_pred = tf.matmul(x ,W)
y_true = tf.placeholder(tf.float32, [None, 10])

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                      tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimze)
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [5]:
## time
with tf.Session() as sess:
      # 학습
        sess.run(tf.global_variables_initializer())
        
        for step in range(NUM_STEPS):
            batch_xs, batch_ys = sess.run(next_batch)
            sess.run(gd_step, feed_dict={x: batch_xs['image'], y_true: batch_ys})
            
            if(step + 1) % 100 == 0:
                print(step+1, '|', sess.run(cross_entropy, feed_dict={x: batch_xs['image'], y_true: batch_ys}))
        
        # 테스트
        ans = sess.run(accuracy, feed_dict={x: test_x, y_true: test_y})
        
print("Accuracy: {:.2f}%".format(ans*100))

100 | 0.4886922
200 | 0.39319777
300 | 0.333915
400 | 0.37393627
500 | 0.4054242
600 | 0.2803463
700 | 0.37861168
800 | 0.17819062
900 | 0.21609864
1000 | 0.30032343
Accuracy: 91.72%
